In [1]:
# 이번에는 현재 들어온 인풋값만이 아니라 과거의 데이터와 현재의 데이터 모두를 바탕으로
# 판단하는 Recurrent Neural Network를 만들어보겠습니다.
# cs231n 강의에도 나오는 "Hello"라는 스트링에서 "Hell"이 들어오면 "ello"라는
# 결과값을 만들어내는 RNN 코드를 짜보겠습니다.

In [2]:
# 먼저 Numpy와 Tensorflow를 불러옵니다.

import numpy as np
import tensorflow as tf

In [3]:
# 그리고 우리가 학습시키고자하는 단어 "Hello"를 불러와서 여기 나오는 알파벳들을 
# dictionary 형태로 저장해 놓습니다
# look_up_table은 나중에 인덱스로 찾기 편하게 이를 리스트로 바꾼것입니다.

input_string="hello"
string_to_char=list(input_string)
set_of_char=set(string_to_char)
look_up_table=list(set_of_char)

dict_char={ c:idx for idx,c in enumerate(set_of_char)}
print(dict_char)
print(look_up_table)

{'e': 0, 'l': 2, 'h': 1, 'o': 3}
['e', 'h', 'l', 'o']


In [4]:
# 인풋인 hell과 아웃풋인 ello 역시 만들어줍니다.
x1=[dict_char[i] for i in input_string[:-1]]
y1=[dict_char[i] for i in input_string[1:]]
length=len(x1)

print(x1,y1)

[1, 0, 2, 2] [0, 2, 2, 3]


In [5]:
# 위에서는 인덱스였지만 인풋으로 넣을때는 one-hot encoding으로 바꿔줍니다
# 이렇게 바꾸는 이유는 l이 0이고 o는 3인것이 수치적으로 의미가 없고 그냥 어쩌다보니
# 저렇게 관계가 정해진것이기 때문입니다.
# 인덱스가 들어왔을때 one-hot으로 바꿔주는 함수를 만듭니다.

def index_to_onehot(x,length):
	arr=[]
	for i in x:
		new_arr=[int(i==j) for j in range(length)]
		arr.append(new_arr)
	return arr

In [6]:
# 이제 인풋과 아웃풋을 지정해줍니다.
# 인풋같은 경우는 one-hot으로 바꿔주고 output은 tf.argmax로 인덱스를 추출하면
# 되기 때문에 그대로 전달해줍니다.

input_x =np.array(index_to_onehot(x1,length),dtype=np.float32)
output_y_ =y1

print(input_x)
print(output_y_)

[[ 0.  1.  0.  0.]
 [ 1.  0.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 0.  0.  1.  0.]]
[0, 2, 2, 3]


In [7]:
# 이제 RNN 모델을 만들고 학습시킬때 필요한 인자들을 지정해봅시다
# 각 인자에 대한 설명은 한칸 밑에서 하도록 하겠습니다.

num_units=length
batch_size=1
learning_rate=1e-2

print(num_units)

4


In [8]:
# RNN셀을 쓸때 요즘에는 LSTM이 기본이지만 예제이기 때문에 기본 RNN 셀을 사용하겠습니다.
# LSTM에 대해 궁금하신분은 여기서 보시면 됩니다 
# http://colah.github.io/posts/2015-08-Understanding-LSTMs/
# 기본 RNN은 텐서플로우 홈페이지에 다음과 같이 정의되어 있습니다
# tf.nn.rnn_cell.BasicRNNCell.__init__(num_units, input_size=None, activation=tanh) 
# 여기서 num_units는 그냥 뉴럴넷에서 히든레이어 노드 수라고 보면됩니다
# tf.RNN에서 hidden layer == output으로 보기 때문에 지금 예시같은 경우에는 
# h,e,l,o니까 4개 또는 length입니다

rnn_cell=tf.nn.rnn_cell.BasicRNNCell(num_units=num_units)

In [9]:
# 또한 RNN은 t=0을 시작이라고 했을때 t=-1일때의 initial state가 필요하기 때문에
# 셀의 크기에 맞춰 다 0으로 초기화 해놓습니다
# 또한 지금 input_x는 4x4 행렬인데 이를 1x4씩 시간에 따라 넣어줘야하기 때문에
# tf.split(split_dim, num_split, value, name='split')
# split_dim 기준으로 num_split 등분해주는 함수입니다

hidden_state_initial=rnn_cell.zero_state(batch_size,dtype=tf.float32)
input_x_split=tf.split(0,length,input_x)

print(input_x_split)

[<tf.Tensor 'split:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:1' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:2' shape=(1, 4) dtype=float32>, <tf.Tensor 'split:3' shape=(1, 4) dtype=float32>]


In [10]:
# RNN 모델에 관한 설명은 아래의 주소에 있습니다. 아직 api에는 없어서 찾기 힘들었습니다
# https://github.com/ahangchen/TensorFlowDoc/blob/master/api_docs/python/functions_and_classes/shard0/tf.nn.rnn.md
# tf.nn.rnn(cell, inputs, initial_state=None, dtype=None, sequence_length=None, scope=None)

outputs, state = tf.nn.rnn(rnn_cell, input_x_split, hidden_state_initial)

print(outputs)
print(state)

[<tf.Tensor 'RNN/BasicRNNCell/Tanh:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_1/Tanh:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_2/Tanh:0' shape=(1, 4) dtype=float32>, <tf.Tensor 'RNN/BasicRNNCell_3/Tanh:0' shape=(1, 4) dtype=float32>]
Tensor("RNN/BasicRNNCell_3/Tanh:0", shape=(1, 4), dtype=float32)


In [11]:
# 이렇게 나온 output을 가지고 loss를 계산하고 backpropagate해야 하는데 
# 그냥 numpy만 가지고 하면 상당히 복잡합니다
# 텐서플로우는 친절하게 이러한 경우에도 자동으로 backprop되게 구현을 해놓았는데
# 이때 사용하는 함수가 tf.nn.seq2seq.sequence_loss_by_example()입니다
# tf.contrib.legacy_seq2seq.sequence_loss_by_example(logits, targets, weights, average_across_timesteps=True, softmax_loss_function=None, name=None) 
# https://github.com/tensorflow/tensorflow/blob/287db3a9b0701021f302e7bb58af5cf89fdcd424/tensorflow/g3doc/api_docs/python/functions_and_classes/shard5/tf.contrib.legacy_seq2seq.sequence_loss_by_example.md#tfcontriblegacy_seq2seqsequence_loss_by_examplelogits-targets-weights-average_across_timestepstrue-softmax_loss_functionnone-namenone-sequence_loss_by_example
# 여기에 들어가서 설명을 읽어보면 logits은 [batch_size x num_decoder_symbols]
# targets는 logits와 같은 길이의 1차원 텐서, weights는 각 time 별로의 가중치입니다
# 상당히 복잡하기 때문에 한가지씩 설명해드리겠습니다.

In [12]:
# 먼저 logits입니다. RNN에서 돌린 결과는 각각 time별로의 output이고 예시에서는 1x4짜리 배열입니다
# 이 결과들을 [batch x num_decoder_symbols]로 바꾸려면 일단 output들을 concatenate해야합니다
# 그 후에는 num_decoder_symbols에 맞춰 reshape 해줍니다. 
# 여기서 batch는 time에 따라 들어오는 인풋을 의미하기 때문에 결과적으로 logit은 4x4가 됩니다

logits = tf.reshape(tf.concat(1, outputs), [-1, num_units])
print(logits)

Tensor("Reshape:0", shape=(4, 4), dtype=float32)


In [13]:
# targets 역시 length에 맞춰 reshape 해줍니다 
# 이 코드는 돌려도 형태에는 변화가 없습니다

targets = tf.reshape(output_y_, [-1])
print(targets)

Tensor("Reshape_1:0", shape=(4,), dtype=int32)


In [14]:
# weights는 time별 가중치인데 여기서는 모두 같게 주기 때문에
# 편하게 tf.ones를 사용했습니다.

weights = tf.ones([length* batch_size])
print(weights)

Tensor("ones:0", shape=(4,), dtype=float32)


In [15]:
# 이제 tf.nn.seq2seq.sequence_loss_by_example를 사용해 loss를 계산해줍니다
# []을 해주는 이유는 logits,targets,weights가 각각 array이기 때문에 
# 콤마만으로는 어디서부터 어디까지가 경계인지 구분할 수 없기도 하고 
# 함수 내부에서 크기 비교를 텐서가 아닌 파이썬 배열로 해놓았기 때문입니다. 
# loss를 통해 cost를 구하고 optimizer도 지정해 줍니다

loss = tf.nn.seq2seq.sequence_loss_by_example([logits], [targets], [weights])
cost = tf.reduce_sum(loss) / batch_size
train_op = tf.train.AdamOptimizer(learning_rate).minimize(cost)

In [16]:
# 드디어 세션을 돌릴 수 있게 되었습니다.
# 처음에는 이상한 단어를 예상하다가 점점 알맞은 단어를 예측하게 학습되었음을 볼 수 있습니다
# 맨 위에 input_string을 바꾸면서 다른 단어에 대해서도 테스트 할 수 있습니다.

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(100):
        sess.run(train_op)
        result = sess.run(tf.argmax(logits, 1))
        print(result)
    print(result, [look_up_table[t] for t in result])


[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 0]
[1 0 2 3]
[1 0 2 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 0 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 3 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
[0 2 2 3]
